# PROGETTO TEAM 24
Christan Internò 876238, Davide Miori 813692, Francesco Oliviero 812292

In [1]:
import pandas as pd
import re

### 0. Preprocessing

Importo i tre datatset e verifico la presenza di missing values

In [2]:
cwur = pd.read_csv("cwurData.csv")
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


Modifico il nome della colonna Institution del dataset cwur in university_name al fine di renderlo compatibile agli altri due

In [3]:
cwur.rename(columns={'institution':'university_name'}, inplace=True)
cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


Noto che i missing sono presenti solo in una variabile che per il momento non è interessante quindi non li rimuovo

In [4]:
cwur.isnull().sum()

world_rank                0
university_name           0
country                   0
national_rank             0
quality_of_education      0
alumni_employment         0
quality_of_faculty        0
publications              0
influence                 0
citations                 0
broad_impact            200
patents                   0
score                     0
year                      0
dtype: int64

In [5]:
times = pd.read_csv("timesData.csv")
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


Diverse colonne con missing che al momento non sono importanti perciò non rimuovo nulla

In [6]:
times.isnull().sum()

world_rank                  0
university_name             0
country                     0
teaching                    0
international               0
research                    0
citations                   0
income                      0
total_score                 0
num_students               59
student_staff_ratio        59
international_students     67
female_male_ratio         233
year                        0
dtype: int64

In [7]:
shanghai = pd.read_csv("shanghaiData.csv")
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [8]:
shanghai.isnull().sum()

world_rank            0
university_name       1
national_rank         1
total_score        3796
alumni                1
award                 2
hici                  2
ns                   22
pub                   2
pcp                   2
year                  0
dtype: int64

Rimuovo la riga non presentante il nome dell'università poichè inutile per la risoluzione dei primi punti

In [9]:
shanghai[shanghai['university_name'].isnull()==True]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
3896,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


Verifico che la rimozione abbia avuto buon fine

In [10]:
shanghai.drop([3896],inplace = True)

In [11]:
shanghai[shanghai['university_name'].isnull()==True]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year


Resetto l'index

In [12]:
shanghai.reset_index(drop=True,inplace=True)

### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

Seleziono prima l'anno minore poi quello maggiore dove aver raggruppato per il nome delle università

In [13]:
times_min = times.iloc[times.groupby('university_name')['year'].idxmin()]
times_min.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


In [14]:
times_max = times.iloc[times.groupby('university_name')['year'].idxmax()]
times_max.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

Osservo che la colonna income presenta dei valori anomali

In [15]:
times_max[times_max['income'].str.contains('-')]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2303,501-600,American University of Beirut,Lebanon,27.7,93.0,11.2,31.9,-,-,"7,695",8.9,25%,51 : 49,2016
2312,501-600,Blaise Pascal University,France,20.2,66.6,2.9,46.9,-,-,"15,619",7.1,17%,55 : 45,2016
2432,601-800,"California State University, Long Beach",United States of America,11.6,18.9,10.4,23.3,-,-,"29,594",32.3,5%,58 : 42,2016
1817,15,Columbia University,United States of America,85.9,73.5,82.2,98.1,-,86.1,"25,055",5.9,28%,NaN,2016
2462,601-800,Federal University of Bahia,Brazil,18.2,19.7,8.1,14.3,-,-,"39,861",17.9,1%,47 : 53,2016
2469,601-800,Federal University of Lavras,Brazil,25.8,14.5,16.4,3.0,-,-,"10,350",20.9,2%,53 : 47,2016
2464,601-800,Federal University of Paraná (UFPR),Brazil,24.6,15.5,9.6,7.0,-,-,"21,850",11.2,1%,NaN,2016
2465,601-800,Federal University of Rio Grande do Sul,Brazil,32.6,18.9,13.7,16.8,-,-,"38,743",14.8,1%,50 : 50,2016
2467,601-800,Federal University of São Carlos,Brazil,29.0,16.8,10.8,11.5,-,-,"18,587",16.9,0%,NaN,2016


Considero solo le righe che non contengono nella variabile income il simbolo '-'

In [16]:
times_max_rm = times_max[times_max['income'].str.contains('-')==False][['university_name','income','year']]

In [17]:
times_min_rm = times_min[times_min['income'].str.contains('-')==False][['university_name','income','year']]

Prima di operare la sottrazione converto i valori di income in numerici 'float'.

In [18]:
times_max_rm['income'] = times_max_rm['income'].astype(float)
times_min_rm['income'] = times_min_rm['income'].astype(float)
times_max_rm.head()

,university_name,income,year
2003,Aalborg University,43.7,2016
2056,Aalto University,61.6,2016
1908,Aarhus University,68.3,2016
2105,Aberystwyth University,31.3,2016
2404,Adam Mickiewicz University,28.7,2016


Verifico che la conversione sia andata a buon fine

In [19]:
times_max_rm.dtypes

university_name     object
income             float64
year                 int64
dtype: object

Uniamo i due dataset  per mezzo della colonna delle università selezionando solo le colonne desiderate

In [20]:
merged = pd.merge(times_max_rm, times_min_rm, on='university_name', suffixes = ['_mostrecent', '_leastrecent'])
merged.head()

,university_name,income_mostrecent,year_mostrecent,income_leastrecent,year_leastrecent
0,Aalborg University,43.7,2016,36.4,2012
1,Aalto University,61.6,2016,61.9,2012
2,Aarhus University,68.3,2016,61.5,2011
3,Aberystwyth University,31.3,2016,35.5,2012
4,Adam Mickiewicz University,28.7,2016,28.7,2016


Effettuo la differenza tra l'income dell'anno più recente rispetto a quello più remoto.

In [21]:
merged['difference'] = merged['income_mostrecent'] - merged['income_leastrecent']
merged.head()

,university_name,income_mostrecent,year_mostrecent,income_leastrecent,year_leastrecent,difference
0,Aalborg University,43.7,2016,36.4,2012,7.3
1,Aalto University,61.6,2016,61.9,2012,-0.3
2,Aarhus University,68.3,2016,61.5,2011,6.8
3,Aberystwyth University,31.3,2016,35.5,2012,-4.2
4,Adam Mickiewicz University,28.7,2016,28.7,2016,0.0


### 3. Find the university with the largest increase computed in the previous point

Ordino le righe di merged rispettivamente alla differenza in senso decrescente.

In [22]:
merged.sort_values(by='difference', ascending=False)

,university_name,income_mostrecent,year_mostrecent,income_leastrecent,year_leastrecent,difference
373,TU Dresden,99.7,2016,31.9,2012,67.8
238,Nanyang Technological University,99.9,2016,40.0,2011,59.9
198,LMU Munich,100.0,2016,40.4,2011,59.6
173,Karlsruhe Institute of Technology,99.5,2016,40.0,2011,59.5
236,Nagoya University,91.4,2016,33.1,2012,58.3
...,...,...,...,...,...,...
380,Technical University of Denmark,50.0,2016,95.5,2011,-45.5
508,University of Iceland,28.0,2016,75.4,2012,-47.4
205,Leiden University,49.8,2016,100.0,2011,-50.2
430,University of Arizona,32.4,2016,84.2,2011,-51.8


### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

Noto che nella colonna world_rank di Times sono presenti degli '='.

In [24]:
times[times['world_rank'].str.contains('=')==True]['world_rank'].head()

1841    =39
1842    =39
1846    =44
1847    =44
1849    =47
Name: world_rank, dtype: object

Rimuovo il simbolo '=' sostituendolo con uno spazio vuoto.

In [25]:
times['world_rank_norm']=times['world_rank'].str.replace('=','')

Verifico di aver rimosso correttamente.

In [26]:
times[times['world_rank_norm'].str.contains('=')==True]['world_rank_norm'].head()

Series([], Name: world_rank_norm, dtype: object)

Noto che sia il dataset shanghai sia times hanno un range di valori nel world_rank come anticipato dalla consegna. Decido dunque di considerare il valore medio ai fini dell'analisi.

In [27]:
times['world_rank']

0             1
1             2
2             3
3             4
4             5
         ...   
2598    601-800
2599    601-800
2600    601-800
2601    601-800
2602    601-800
Name: world_rank, Length: 2603, dtype: object

Definisco la funzione per cacolare tale valore medio.

In [28]:
def mean_calculator(valore):
    if len(valore)==7:
        
        pattern = re.compile("(\d+)\-(\d+)")
        occurrence = pattern.search(valore)
        
        if occurrence:
            primo = int(occurrence.group(1))
            secondo = int(occurrence.group(2))
            media = (primo + secondo) / 2
            return int(media)
        
    if len(valore) != 7:
        return int(valore)

Applico la funzione alle colonne dei due dataset sopra citati.

In [29]:
times['world_rank_norm'] = times['world_rank_norm'].apply(mean_calculator)

Verifico che la media sia stata calcolata correttamente

In [30]:
times.loc[1764]

world_rank                                              351-400
university_name           Indian Institute of Technology Bombay
country                                                   India
teaching                                                   34.2
international                                              19.5
research                                                   24.6
citations                                                  43.5
income                                                     49.7
total_score                                                   -
num_students                                              8,327
student_staff_ratio                                        14.9
international_students                                       1%
female_male_ratio                                       16 : 84
year                                                       2015
world_rank_norm                                             375
Name: 1764, dtype: object

Applico la funzione anche a shanghai.

In [31]:
shanghai['world_rank_norm'] = shanghai['world_rank'].apply(mean_calculator)

In [32]:
shanghai.loc[1981]

world_rank                        402-503
university_name    University of Hannover
national_rank                       36-40
total_score                           NaN
alumni                                  0
award                                   0
hici                                  7.3
ns                                    6.2
pub                                    25
pcp                                  12.3
year                                 2008
world_rank_norm                       452
Name: 1981, dtype: object

Seleziono gli anni più recenti per i tre dataset.

In [33]:
cwur_ranking = cwur.iloc[cwur.groupby('university_name')['year'].idxmax()].copy()
cwur_ranking.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015


In [34]:
shanghai_ranking = shanghai.iloc[shanghai.groupby('university_name')['year'].idxmax()].copy()
shanghai_ranking.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm
4696,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,350
4796,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,450
4468,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73
4496,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015,125
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,126


In [35]:
times_ranking = times.iloc[times.groupby('university_name')['year'].idxmax()].copy()
times_ranking.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,700
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,225
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,275
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,325


Importo il dataset school_and_country poichè contiene i nomi normalizzati delle università per paese.

In [36]:
school = pd.read_csv('school_and_country_table.csv')
school.head()

,school_name,country
0,Harvard University,United States of America
1,California Institute of Technology,United States of America
2,Massachusetts Institute of Technology,United States of America
3,Stanford University,United States of America
4,Princeton University,United States of America


Scrivo una funzione base per normalizzare i nomi delle università

In [37]:
def norm_function (value):
    
    for university in school['school_name']:
        if value == university:
            return value
        
    pattern = re.compile(value)
    for university in school['school_name']:
        occurrence = pattern.search(university)
        if occurrence:
            return university
        else:
            pattern2 = re.compile(university)
            occurrence2 =pattern2.search(value)
            if occurrence2:
                return university

Applico la funzione a shanghai.

In [38]:
shanghai_ranking['school_name'] = shanghai_ranking['university_name'].apply(norm_function)

Verifico se si sono generati dei doppioni.

In [39]:
shanghai_ranking[(shanghai_ranking['school_name'].isnull()==False) & (shanghai_ranking['school_name'].duplicated()==True)]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name
3876,79,Arizona State University - Tempe,46.0,26.1,0.0,20.1,27.1,27.5,42.5,18.9,2013,79,Arizona State University
3425,401-500,Curtin University of Technology,14-19,NaN,0.0,0.0,0.0,10.5,29.4,16.3,2011,450,Curtin University
925,401-500,Louisiana State University Health Sciences Center,141-167,NaN,0.0,0.0,7.7,0.0,31.0,14.6,2006,450,Louisiana State University
4509,101-150,Norwegian University of Science and Technology...,2,NaN,13.6,21.0,10.1,12.9,36.4,20.0,2015,125,Norwegian University of Science and Technology
4457,61,Purdue University - West Lafayette,37,29.2,13.6,23.1,27.7,25.1,49.0,22.4,2015,61,Purdue University
4132,201-300,Royal Institute of Technology,6-8,NaN,0.0,14.9,3.6,9.9,38.0,23.5,2014,250,KTH Royal Institute of Technology
3606,93,Texas A&M University - College Station,51,24.6,0.0,0.0,33.9,22.5,53.2,19.2,2012,93,Texas A&M University
3912,17,The Johns Hopkins University,15,47.0,38.7,33.6,44.0,47.8,70.2,28.7,2014,17,Johns Hopkins University
3461,401-500,The University of Connecticut Health Center,138-151,NaN,0.0,0.0,10.2,7.4,21.7,11.6,2011,450,University of Connecticut
4568,151-200,The University of Hong Kong,1-2,NaN,0.0,0.0,17.5,14.2,43.9,22.5,2015,175,University of Hong Kong


In [40]:
a = shanghai_ranking.groupby('school_name').size()
a[a>1]

school_name
Arizona State University                          2
Curtin University                                 2
Johns Hopkins University                          2
KTH Royal Institute of Technology                 2
Louisiana State University                        2
Norwegian University of Science and Technology    2
Purdue University                                 2
State University of Campinas                      2
Texas A&M University                              2
University of Arkansas                            2
University of Bordeaux                            2
University of Connecticut                         3
University of Graz                                2
University of Hong Kong                           2
University of Innsbruck                           2
University of Kansas                              3
University of Maryland, Baltimore County          2
University of Massachusetts                       2
University of Melbourne                           2


In [41]:
shanghai_ranking[shanghai_ranking['school_name']=='Curtin University']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name
4605,201-300,Curtin University,9-11,NaN,0.0,0.0,8.7,12.0,37.1,23.3,2015,250,Curtin University
3425,401-500,Curtin University of Technology,14-19,NaN,0.0,0.0,0.0,10.5,29.4,16.3,2011,450,Curtin University


In [43]:
shanghai_ranking[shanghai_ranking['school_name']=='University of Arkansas']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name
4852,401-500,University of Arkansas at Fayetteville,126-146,NaN,0.0,0.0,12.2,6.1,24.4,14.1,2015,450,University of Arkansas
4853,401-500,University of Arkansas at Little Rock,126-146,NaN,0.0,0.0,10.7,2.1,24.4,19.9,2015,450,University of Arkansas


In [44]:
shanghai_ranking[shanghai_ranking['school_name']=='University of Milan']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name
4584,151-200,University of Milan,1-5,NaN,17.0,0.0,14.2,9.6,46.3,21.2,2015,175,University of Milan
4872,401-500,University of Milan - Bicocca,11-20,NaN,0.0,0.0,5.0,5.6,30.9,21.4,2015,450,University of Milan


In [45]:
shanghai_ranking[shanghai_ranking['school_name']=='University of Wisconsin']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name
4419,24,University of Wisconsin - Madison,18,41.1,30.8,34.8,40.2,35.7,62.5,24.6,2015,24,University of Wisconsin
3501,401-500,University of Wisconsin - Milwaukee,138-151,NaN,11.8,0.0,0.0,6.8,25.7,12.6,2011,450,University of Wisconsin


In [46]:
school[school['school_name']=='University of Wisconsin-Milwaukee']

,school_name,country
630,University of Wisconsin-Milwaukee,United States of America


Rimuovo la colonna normalizzata.

In [47]:
del shanghai_ranking['school_name']

Verifico di averla rimossa

In [48]:
shanghai_ranking.head(1)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm
4696,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,350


Rimuovo le righe presentanti dei valori duplicati dovuti alla scorretta nominazione dell'università.

In [49]:
shanghai_ranking.drop([3425, 3738, 4132,3606,3912,3383,3748,3783,3853,3820,4249,3654,4274,3860,3960,3309], inplace=True)

Scrivo una funzione da applicare esclusivamente a shanghai in quanto alcune problematiche sono sorte solamente in merito ad essa. Tale funzione normalizza le università escludendo i campus non rappresentati la facoltà nella sua interezza. Inoltre, alcune scuole differenti risultano appartenere alla stessa università (es Milano-Bicocca).

In [50]:
def norm_shanghai (value):
    
    
    if value in ['Arizona State University - Tempe','Louisiana State University Health Sciences Center', 'Indiana University-Purdue University at Indianapolis',
                'The University of Connecticut - Storrs','The University of Connecticut Health Center',
                 'University of Arkansas at Little Rock',  'Medical University of Graz',
                 'Medical University of Innsbruck','University of Kansas Medical Center',
                  'University of Maryland, Baltimore','University of Massachusetts Medical School - Worcester']:
        return None
    
    
    for university in school['school_name']:
        if value == university:
            return value
    
        if value == 'The Chinese University of Hong Kong' and university == 'Chinese University of Hong Kong':
            return university
        
        if value == 'University of Milan - Bicocca' and university == 'University of Milan-Bicocca':
            return university
        
        if value == 'University of Wisconsin - Madison' and university == 'University of Wisconsin-Madison':
            return university
        
        if value == 'University of Wisconsin - Milwaukee' and university == 'University of Wisconsin-Milwaukee':
            return university
        
    
    pattern = re.compile(value)
    for university in school['school_name']:
        occurrence = pattern.search(university)
        if occurrence:
            return university
        else:
            pattern2 = re.compile(university)
            occurrence2 = pattern2.search(value)
            if occurrence2:
                return university          
    

Applico la funzione a shanghai.

In [51]:
shanghai_ranking['school_name'] = shanghai_ranking['university_name'].apply(norm_shanghai)

Verifico nuovamente la presenza di valori duplicati nella colonna normalizzata.

In [52]:
shanghai_ranking[(shanghai_ranking['school_name'].isnull()==False) & (shanghai_ranking['school_name'].duplicated()==True)]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,school_name


Normalizzo Times.

In [53]:
times_ranking['school_name'] = times_ranking['university_name'].apply(norm_function)

In [54]:
times_ranking[(times_ranking['school_name'].isnull()==False) & (times_ranking['school_name'].duplicated()==True)]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name


Normalizzo Cwur il quale presenta come per shanghai alcune problematiche.

In [55]:
cwur_ranking['school_name'] = cwur_ranking['university_name'].apply(norm_function)

Ottengo come in precedenza dei duplicati.

In [56]:
cwur_ranking[(cwur_ranking['school_name'].isnull()==False) & (cwur_ranking['school_name'].duplicated()==True)]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,school_name
2094,895,Nanjing University of Aeronautics and Astronau...,China,61,367,486,218,709,974,812,867.0,853,44.14,2015,Nanjing University
2003,804,Nanjing University of Science and Technology,China,48,367,331,218,796,943,368,826.0,694,44.24,2015,Nanjing University
2009,810,Nanjing University of Technology,China,49,367,567,218,748,984,812,932.0,217,44.22,2015,Nanjing University
2115,916,National Taiwan University of Science and Tech...,Taiwan,15,367,567,218,598,991,812,886.0,823,44.12,2015,National Taiwan University
2070,871,Northeastern University (China),China,55,367,315,218,759,974,812,906.0,698,44.16,2015,Northeastern University
1242,43,"Purdue University, West Lafayette",USA,27,55,33,47,75,95,68,117.0,28,56.39,2015,Purdue University
2168,969,South China Agricultural University,China,73,367,567,218,877,811,812,932.0,871,44.06,2015,China Agricultural University
1476,277,Sun Yat-sen University,China,8,367,254,218,161,393,287,285.0,221,46.25,2015,Sun Yat-sen University
1628,429,University of Arkansas for Medical Sciences,USA,143,367,567,218,501,565,812,344.0,340,45.08,2015,University of Arkansas
631,432,University of Bordeaux I,France,17,355,478,210,327,228,406,361.0,426,45.41,2014,University of Bordeaux


Procedo come in precedenza.

In [57]:
cwur_ranking[cwur_ranking['school_name']=='University of Missouri']

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,school_name
1393,194,University of Missouri–Columbia,USA,84,250,88,98,222,226,161,242.0,150,47.59,2015,University of Missouri
1753,554,University of Missouri–Kansas City,USA,169,311,480,218,634,447,645,471.0,687,44.61,2015,University of Missouri
1947,748,University of Missouri–St. Louis,USA,203,367,371,218,897,524,428,715.0,634,44.30,2015,University of Missouri


In [58]:
cwur_ranking[cwur_ranking['school_name']=='University of Bordeaux']

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,school_name
1498,299,University of Bordeaux,France,14,354,567,218,291,265,234,251.0,297,45.96,2015,University of Bordeaux
631,432,University of Bordeaux I,France,17,355,478,210,327,228,406,361.0,426,45.41,2014,University of Bordeaux
602,403,University of Bordeaux II,France,15,355,478,210,440,385,363,322.0,426,45.54,2014,University of Bordeaux


In [59]:
del cwur_ranking['school_name']

In [60]:
cwur_ranking.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015


Scrivo la funzione personalizzata per cwur in cui indico i campus secondari o non presenti nel dataset school_country infatti University of Massachussets si presenta in cwur in diverse città. Tuttavia, solo quella residente in Amherst è rappresentativa dello stato. Per questa ragione si decide di non considerare le altre. 
Inoltre, avremmo potuto aggiungere alcune righe al datatset school_country riferite a tutte le univeristà del Massachussets al fine di trovare un collegamento con il dataset cwur e mantenerle nell'analisi.

In [61]:
def norm_world (value):
    
    
    if value in ['University of Bordeaux I', 'University of Bordeaux II', 'Nanjing University of Aeronautics and Astronautics','Nanjing University of Science and Technology','Nanjing University of Technology',
                  'Northeastern University (China)', 'Indiana University-Purdue University Indianapolis',                            
                    'South China Agricultural University','University of Arkansas for Medical Sciences',
                      'Medical University of Graz', 'University of Maryland, Baltimore','University of Massachusetts Boston',
                         'University of Massachusetts Lowell','University of Massachusetts Medical School', 'University of Missouri–Kansas City',
                            'University of Missouri–St. Louis','University of Oklahoma Health Sciences Center',
                            'Federal University of São Paulo','Zhejiang University of Technology', 'École Polytechnique de Montréal']:
        return None
    
    
    for university in school['school_name']:
        if value == university:
            return value
        
        
        if value == 'National Taiwan University of Science and Technology'\
        and university == 'National Taiwan University of Science and Technology (Taiwan Tech)':
            return university
        
        if value == 'National Sun Yat-sen University' and university == 'National Sun Yat-Sen University':
            return university
        
        if value == 'University of Milan - Bicocca' and university == 'University of Milan-Bicocca':
            return university
        
        if value == 'University of Wisconsin–Madison' and university == 'University of Wisconsin-Madison':
            return university
        
        if value == 'University of Wisconsin–Milwaukee' and university == 'University of Wisconsin-Milwaukee':
            return university
        
        
        
    pattern = re.compile(value)
    for university in school['school_name']:
        occurrence = pattern.search(university)
        if occurrence:
            return university
        else:
            pattern2 = re.compile(university)
            occurrence2 =pattern2.search(value)
            if occurrence2:
                return university
            
    

In [62]:
cwur_ranking['school_name'] = cwur_ranking['university_name'].apply(norm_world)

In [63]:
cwur_ranking[(cwur_ranking['school_name'].isnull()==False) & (cwur_ranking['school_name'].duplicated()==True)]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,school_name


Non considero le righe presentati missing values nella colonna generata con la funzione.

In [64]:
sh_final = shanghai_ranking[shanghai_ranking['school_name'].isnull()==False][['school_name','world_rank_norm']]
sh_final.head()

,school_name,world_rank_norm
4696,Aalborg University,350
4796,Aalto University,450
4468,Aarhus University,73
3115,Aix-Marseille University,126
4797,Aristotle University of Thessaloniki,450


In [65]:
cwur_final = cwur_ranking[cwur_ranking['school_name'].isnull()==False][['school_name','world_rank']]


In [66]:
tm_final = times_ranking[times_ranking['school_name'].isnull()==False][['school_name','world_rank_norm']]


Unisco i tre dataset, due per volta, mediante la colonna normalizzata.

In [67]:
first_merge= pd.merge(sh_final, tm_final, on='school_name')
first_merge.head()

,school_name,world_rank_norm_x,world_rank_norm_y
0,Aalborg University,350,225
1,Aalto University,450,275
2,Aarhus University,73,106
3,Aix-Marseille University,126,275
4,Aristotle University of Thessaloniki,450,700


In [68]:
final_merge = pd.merge(first_merge, cwur_final, on='school_name')
final_merge.head()

,school_name,world_rank_norm_x,world_rank_norm_y,world_rank
0,Aalborg University,350,225,565
1,Aalto University,450,275,421
2,Aarhus University,73,106,122
3,Aix-Marseille University,126,275,206
4,Aristotle University of Thessaloniki,450,700,459


Modifico i nomi delle colonne per chiarezza.

In [69]:
final_merge.rename(columns={'world_rank_norm_x':'shanghai_ranking', 'world_rank_norm_y':'times_ranking', 'world_rank':'cwur_ranking'},inplace=True)
final_merge.head()

,school_name,shanghai_ranking,times_ranking,cwur_ranking
0,Aalborg University,350,225,565
1,Aalto University,450,275,421
2,Aarhus University,73,106,122
3,Aix-Marseille University,126,275,206
4,Aristotle University of Thessaloniki,450,700,459


Scrivo una funzione per calcolare la differenza richiesta dal punto 4.

In [70]:
def max_min_difference(value):
    maximum = max(value['shanghai_ranking'],value['times_ranking'], value['cwur_ranking'])
    minimum = min(value['shanghai_ranking'],value['times_ranking'], value['cwur_ranking'])
    
    return (maximum - minimum)

In [71]:
final_merge['difference'] = final_merge.apply(max_min_difference,axis=1)
final_merge

,school_name,shanghai_ranking,times_ranking,cwur_ranking,difference
0,Aalborg University,350,225,565,340
1,Aalto University,450,275,421,175
2,Aarhus University,73,106,122,49
3,Aix-Marseille University,126,275,206,149
4,Aristotle University of Thessaloniki,450,700,459,250
...,...,...,...,...,...
431,Yale University,11,12,11,1
432,Yeshiva University,250,164,171,86
433,Yonsei University,250,325,98,227
434,York University,450,325,337,125


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

D'ora in poi creerò una copia per ogni dataset utilizzato e per ogni punto del problema.

In [72]:
times5 = times_ranking.copy()
times5.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,700,AGH University of Science and Technology
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,225,Aalborg University
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,275,Aalto University
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106,Aarhus University
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,325,Aberystwyth University


Noto che nella colonna female_male_ratio sono presenti dei valori '-' che decido di rimuovere.

In [73]:
times5[times5['female_male_ratio'].str.contains('-')==True]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,700,AGH University of Science and Technology
2322,501-600,Czech Technical University in Prague,Czech Republic,20.6,25.0,9.3,46.5,28.2,-,"22,751",12.0,NaN,-,2016,550,Czech Technical University in Prague
2343,501-600,King Fahd University of Petroleum and Minerals,Saudi Arabia,25.4,84.5,16.1,24.4,81.2,-,"6,898",7.1,20%,-,2016,550,King Fahd University of Petroleum and Minerals


In [74]:
times5.drop([2405,2322,2343], inplace=True)

Controllo inoltre nelle colonne di interesse se sono presenti dei missing.

In [75]:
times5['num_students'].isnull().sum()

22

In [76]:
times5['female_male_ratio'].isnull().sum()

76

Decido di rimuoverli e verifico la rimozione.

In [77]:
times5 = times5[times5['num_students'].isnull()==False]
times5[times5['num_students'].isnull()]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name


In [78]:
times5 = times5[times5['female_male_ratio'].isnull()==False]
times5[times5['female_male_ratio'].isnull()]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name


Noto anche che le colonne di interesse non sono numeriche.

In [79]:
times5.dtypes

world_rank                 object
university_name            object
country                    object
teaching                  float64
international              object
research                  float64
citations                 float64
income                     object
total_score                object
num_students               object
student_staff_ratio       float64
international_students     object
female_male_ratio          object
year                        int64
world_rank_norm             int64
school_name                object
dtype: object

Rimuovo la virgola al fine di renderle numeriche.

In [80]:
times5['num_students'] = times5['num_students'].str.replace(',','')

In [81]:
times5['num_students'] = times5['num_students'].astype(float)
times5.dtypes

world_rank                 object
university_name            object
country                    object
teaching                  float64
international              object
research                  float64
citations                 float64
income                     object
total_score                object
num_students              float64
student_staff_ratio       float64
international_students     object
female_male_ratio          object
year                        int64
world_rank_norm             int64
school_name                object
dtype: object

Scrivo una funzione che calcola la percentuale di femmine dal numero totale di studenti. I valori female_male sono resi numerici dalla funzione.

In [82]:
def female_perc_calculator(value):
    
    f = int(re.split(':', value['female_male_ratio'])[0])
    students = value['num_students']
    n_female = round((students*f)/100, 0)
    
    return n_female

La applico quindi creando una nuova colonna.

In [83]:
times5['female_number'] = times5.apply(female_perc_calculator, axis=1)
times5.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name,female_number
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,225,Aalborg University,8363.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,275,Aalto University,5152.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,106,Aarhus University,12903.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,325,Aberystwyth University,4441.0
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,-,40633.0,15.6,1%,71 : 29,2016,700,Adam Mickiewicz University,28849.0


Procedo ugualmente per i maschi.

In [84]:
def male_perc_calculator(value):
    
    m = int(re.split(':', value['female_male_ratio'])[1])
    students = value['num_students']
    n_male = round((students*m)/100, 0)
    
    return n_male

In [85]:
times5['male_number'] = times5.apply(male_perc_calculator, axis=1)
times5.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,school_name,female_number,male_number
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,225,Aalborg University,8363.0,9059.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,275,Aalto University,5152.0,10947.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,106,Aarhus University,12903.0,10992.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,325,Aberystwyth University,4441.0,4811.0
2404,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,-,40633.0,15.6,1%,71 : 29,2016,700,Adam Mickiewicz University,28849.0,11784.0


Raggruppo per country, selezionando alcune colonne di interesse e sommando le varie percentuali.

In [86]:
country_ratio = times5.groupby('country')[['country','female_number','male_number']].sum()
country_ratio

,female_number,male_number
country,,
Argentina,67191.0,41182.0
Australia,391736.0,321640.0
Austria,68364.0,66113.0
Bangladesh,21323.0,41393.0
Belarus,20219.0,9084.0
...,...,...
Uganda,18670.0,18670.0
Ukraine,17846.0,19250.0
United Arab Emirates,9516.0,4931.0


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

Considero solamente le colonne utili ai fini del problema dal dataset del punto precedente prima del raggruppamento.

In [87]:
times6 = times5[['university_name','country','female_number','male_number']].copy()
times6.head()

,university_name,country,female_number,male_number
2003,Aalborg University,Denmark,8363.0,9059.0
2056,Aalto University,Finland,5152.0,10947.0
1908,Aarhus University,Denmark,12903.0,10992.0
2105,Aberystwyth University,United Kingdom,4441.0,4811.0
2404,Adam Mickiewicz University,Poland,28849.0,11784.0


Scrivo un'altra funzione che calcoli il rapporto tra il numero di femmine e maschi.

In [88]:
def compute_ratio(value):
    if  (value['male_number'] == 0):
        return None
    else:
        return round(value['female_number'] / value['male_number'] *100,1)

In [89]:
times6['ratio']=times6.apply(compute_ratio,axis=1)
times6.head()

,university_name,country,female_number,male_number,ratio
2003,Aalborg University,Denmark,8363.0,9059.0,92.3
2056,Aalto University,Finland,5152.0,10947.0,47.1
1908,Aarhus University,Denmark,12903.0,10992.0,117.4
2105,Aberystwyth University,United Kingdom,4441.0,4811.0,92.3
2404,Adam Mickiewicz University,Poland,28849.0,11784.0,244.8


Considero, come da richiesta, solo le università con il rapporto minore del valore medio dello stesso.

In [90]:
univ_below = times6[times6['ratio']<times6['ratio'].mean()]
univ_below

,university_name,country,female_number,male_number,ratio
2003,Aalborg University,Denmark,8363.0,9059.0,92.3
2056,Aalto University,Finland,5152.0,10947.0,47.1
2105,Aberystwyth University,United Kingdom,4441.0,4811.0,92.3
2406,Ajou University,South Korea,4193.0,8513.0,49.3
2408,Alexandria University,Egypt,58618.0,68813.0,85.2
...,...,...,...,...,...
2104,Zhejiang University,China,19478.0,28030.0,69.5
1856,École Normale Supérieure,France,1104.0,1296.0,85.2
2013,École Normale Supérieure de Lyon,France,1087.0,1131.0,96.1
1904,École Polytechnique,France,437.0,1992.0,21.9


### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

Considero il dataset sopra creato.

In [91]:
times7 = univ_below.copy()

Creo una nuova colonna data dalla somma tra le femmine e i maschi.

In [92]:
times7['students_number'] = times7['female_number']+times7['male_number']
times7.head(1)

,university_name,country,female_number,male_number,ratio,students_number
2003,Aalborg University,Denmark,8363.0,9059.0,92.3,17422.0


raggruppo per country sommando i valori.

In [93]:
times7 = times7.groupby('country').sum()
times7.head()

,female_number,male_number,ratio,students_number
country,,,,
Australia,69448.0,76787.0,548.3,146235.0
Austria,21935.0,39098.0,257.5,61033.0
Bangladesh,21323.0,41393.0,51.5,62716.0
Brazil,130413.0,148361.0,747.9,278774.0
Canada,41102.0,45676.0,270.5,86778.0


Considero solo la colonna degli studenti totali per paese

In [94]:
times7 = times7['students_number']
times7.head()

country
Australia     146235.0
Austria        61033.0
Bangladesh     62716.0
Brazil        278774.0
Canada         86778.0
Name: students_number, dtype: float64

Ora, recupero il dataset contenente il numero di maschi e femmine per country. calcolo anche per questo il totale di studenti.

In [95]:
country_ratio['tot_students_number'] = country_ratio['female_number'] + country_ratio['male_number']
country_ratio.head(4)

,female_number,male_number,tot_students_number
country,,,
Argentina,67191.0,41182.0,108373.0
Australia,391736.0,321640.0,713376.0
Austria,68364.0,66113.0,134477.0
Bangladesh,21323.0,41393.0,62716.0


In [96]:
tot_students= country_ratio['tot_students_number']
tot_students.head(2)

country
Argentina    108373.0
Australia    713376.0
Name: tot_students_number, dtype: float64

Unisco i due dataset tramite la colonna dei paesi.

In [97]:
country_merged = pd.merge(times7, tot_students, on='country')
country_merged

,students_number,tot_students_number
country,,
Australia,146235.0,713376.0
Austria,61033.0,134477.0
Bangladesh,62716.0,62716.0
Brazil,278774.0,494251.0
Canada,86778.0,623628.0
Chile,90951.0,90951.0
China,885294.0,1015151.0
Colombia,54645.0,54645.0
Czech Republic,38685.0,175871.0


Calcolo il rapporto tra il numero di studenti riferiti ai due datatset.

In [98]:
country_merged['ratio'] = round((country_merged['students_number']/country_merged['tot_students_number'])*100,2)
country_merged

,students_number,tot_students_number,ratio
country,,,
Australia,146235.0,713376.0,20.50
Austria,61033.0,134477.0,45.39
Bangladesh,62716.0,62716.0,100.00
Brazil,278774.0,494251.0,56.40
Canada,86778.0,623628.0,13.92
Chile,90951.0,90951.0,100.00
China,885294.0,1015151.0,87.21
Colombia,54645.0,54645.0,100.00
Czech Republic,38685.0,175871.0,22.00


### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row with missing `country_name` or `series_name`

Importo il dataset indicato.

In [138]:
educ = pd.read_csv('educational_attainment_supplementary_data.csv')
educ

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Mostro le righe con valori mancanti.

In [137]:
educ[((educ['country_name'].isnull()== True) | (educ['series_name'].isnull()==True))]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eseguo la richiesta.

In [139]:
educ = educ[~((educ['country_name'].isnull()== True) | (educ['series_name'].isnull()==True))]
educ

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

In [100]:
educ.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [331]:
educ.columns

Index(['country_name', 'series_name', '1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'],
      dtype='object')

Utilizzo il comando pd.melt per passare dal formato wide a long quindi comprimendo le colonne degli anni in un'unica e creando un'ulteriore colonna per i valori riferiti agli anni stessi. 

In [140]:
pd.melt(educ, id_vars=['country_name','series_name'], value_vars=['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'], var_name ='year', value_name ='value')

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [102]:
times10 = times.copy()
shanghai10 = shanghai.copy()
cwur10 = cwur.copy()

Per il problema in esame normalizzo nuovamente con le funzioni del punto 4 senza eliminare i duplicati. 

In [104]:
times10['normalized_name'] = times10['university_name'].apply(norm_function)

In [105]:
times10.head(1)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,normalized_name
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,1,Harvard University


In [106]:
shanghai10['normalized_name'] = shanghai10['university_name'].apply(norm_shanghai)

In [107]:
shanghai10.head(1)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_norm,normalized_name
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,1,Harvard University


In [108]:
cwur10['normalized_name'] = cwur10['university_name'].apply(norm_world)

In [109]:
cwur10.head(1)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,normalized_name
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.0,2012,Harvard University


Non considero i missing values.

In [110]:
shanghai10 = shanghai10[shanghai10['normalized_name'].isnull()==False]

In [111]:
times10 = times10[times10['normalized_name'].isnull()==False]
times10.head(1)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,normalized_name
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,1,Harvard University


In [112]:
cwur10 = cwur10[cwur10['normalized_name'].isnull()==False]
cwur10.head(1)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,normalized_name
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.0,2012,Harvard University


Prendo una lista delle università normalizzate per ogni dataset considerando i valori univoci. Dopodichè le concatento in un'unica lista.

In [113]:
times_list = list(times10['normalized_name'].unique())
shanghai_list = list(shanghai10['normalized_name'].unique())
cwur_list = list(cwur10['normalized_name'].unique())

In [114]:
concat_list = times_list + shanghai_list + cwur_list

Importo la funzione Counter che conta quante volte un determinato valore sia presente internamente alla lista creando un dizionario che associa il numero di volte. Lo applico quindi alla lista prima creata.

In [115]:
from collections import Counter

In [116]:
diz=Counter(concat_list)
diz

Counter({'Harvard University': 3,
         'California Institute of Technology': 3,
         'Massachusetts Institute of Technology': 3,
         'Stanford University': 3,
         'Princeton University': 3,
         'University of Cambridge': 3,
         'University of Oxford': 3,
         'University of California, Berkeley': 3,
         'Imperial College London': 2,
         'Yale University': 3,
         'University of California, Los Angeles': 3,
         'University of Chicago': 3,
         'Johns Hopkins University': 3,
         'Cornell University': 3,
         'ETH Zurich – Swiss Federal Institute of Technology Zurich': 2,
         'University of Michigan': 3,
         'University of Toronto': 3,
         'Columbia University': 3,
         'University of Pennsylvania': 3,
         'Carnegie Mellon University': 3,
         'University of Hong Kong': 3,
         'University College London': 3,
         'University of Washington': 3,
         'Duke University': 3,
         'North

### 11. In the times ranking, compute the number of times each university appears

In [117]:
times11 = times.copy()

Prendo una lista delle università nel dataset Times e la ordino alfabeticamente.

In [118]:
university_list = list(times11['university_name'])

In [119]:
university_list.sort()

Applico nuovamente la funzione Counter.

In [120]:
a = Counter(university_list)
a

Counter({'AGH University of Science and Technology': 1,
         'Aalborg University': 5,
         'Aalto University': 5,
         'Aarhus University': 6,
         'Aberystwyth University': 5,
         'Adam Mickiewicz University': 1,
         'Aix-Marseille University': 1,
         'Ajou University': 1,
         'Alexandria University': 3,
         'Alexandru Ioan Cuza University': 1,
         'Aligarh Muslim University': 1,
         'American University': 1,
         'American University of Beirut': 1,
         'American University of Sharjah': 1,
         'Amirkabir University of Technology': 1,
         'Amrita University': 1,
         'Anadolu University': 1,
         'Andhra University': 1,
         'Aristotle University of Thessaloniki': 1,
         'Arizona State University': 6,
         'Asia University, Taiwan': 1,
         'Aston University': 3,
         'Athens University of Economics and Business': 1,
         'Auburn University': 2,
         'Auckland University of Techno

Creo un dataset dal dizionario selezionando solo le colonne util perchè necessario al punto successivo.

In [121]:
tab = pd.DataFrame(a.items(), columns=['university_name', 'number_of_times'])
tab

,university_name,number_of_times
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking.

In [122]:
tab[tab['number_of_times']<=2]

,university_name,number_of_times
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [127]:
times13 = times10.copy()
cwur13 = cwur10.copy()
shanghai13 = shanghai10.copy()

Rimuovo come fatto in precedenza il simbolo uguale.

In [128]:
times13['world_rank']=times13['world_rank'].str.replace('=','')
times13.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_norm,normalized_name
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,1,Harvard University
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011,2,California Institute of Technology
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011,3,Massachusetts Institute of Technology
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011,4,Stanford University
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011,5,Princeton University


Rimuovo anche le righe presentanti il trattino per la colonna world_rank in shanghai e times le quali rappresentano le università con ranking definito come range.

In [129]:
shanghai13 = shanghai13[shanghai13['world_rank'].str.contains('-')==False]
times13 = times13[times13['world_rank'].str.contains('-')==False]

Seleziono le colonne di interesse per i tre dataset.

In [130]:
shanghai_restricted = shanghai13[['normalized_name','year','world_rank']]
times_restricted = times13[['normalized_name','year','world_rank']]
cwur_restricted = cwur13[['normalized_name','year','world_rank']]

Li unisco due a due cambiando i nomi delle colonne.

In [131]:
merge1 = pd.merge(shanghai_restricted, times_restricted, on=['normalized_name','year'])
ultimate_merge = pd.merge(merge1, cwur_restricted,on=['normalized_name','year'])


In [132]:
ultimate_merge.rename(columns={'world_rank_x':'shanghai_ranking','world_rank_y':'times_ranking','world_rank':'cwur_ranking'},inplace = True)
ultimate_merge

,normalized_name,year,shanghai_ranking,times_ranking,cwur_ranking
0,Harvard University,2012,1,2,1
1,Stanford University,2012,2,2,3
2,Massachusetts Institute of Technology,2012,3,7,2
3,"University of California, Berkeley",2012,4,10,10
4,University of Cambridge,2012,5,6,4
...,...,...,...,...,...
283,McMaster University,2015,96,94,150
284,University of Bonn,2015,97,195,158
285,VU University Amsterdam,2015,98,136,129
286,Michigan State University,2015,99,82,118


Converto a numero le colonne su cui devo operare.

In [133]:
ultimate_merge['cwur_ranking'] = ultimate_merge['cwur_ranking'].astype(int)
ultimate_merge['shanghai_ranking'] = ultimate_merge['shanghai_ranking'].astype(int)
ultimate_merge['times_ranking'] = ultimate_merge['times_ranking'].astype(int)
ultimate_merge.dtypes

normalized_name     object
year                 int64
shanghai_ranking     int32
times_ranking        int32
cwur_ranking         int32
dtype: object

Verifico a questro punto la presenza di università con lo stesso ranking per i tre dataset.

In [134]:
ultimate_merge[(ultimate_merge['shanghai_ranking']==ultimate_merge['times_ranking']) & (ultimate_merge['times_ranking']==ultimate_merge['cwur_ranking'])]

,normalized_name,year,shanghai_ranking,times_ranking,cwur_ranking
69,Stanford University,2013,2,2,2


THE END